## Visualisation

À cette étape, nous allons utiliser les bases de données préparées dans l'étape précédente (prepare_data) pour créer une carte qui permet de visualiser les sites de camping et les alentours. Cela nous permettra de former nos premières idées sur les sites de camping (et sur les résultats du clustering), car nous pouvons y voir la disposition de tous les éléments les uns par rapport aux autres.

In [1]:
!pip install pandas fiona shapely pyproj rtree
!pip install geopandas

In [2]:
!pip install pandas-bokeh

In [3]:
import geopandas as gpd
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [4]:
pandas_bokeh.output_file("Visualisation.html")

In [5]:
#Chargement des bases de données préparées
defibrillateurs = pd.read_csv("defibrillateur.csv")
police = pd.read_csv("police.csv")
hopitaux = pd.read_csv("hopitaux.csv")
camping = pd.read_csv("camping.csv")
departements = pd.read_csv("departements.csv")
communes = pd.read_csv("communes.csv")
sans_mobile = pd.read_csv("sans_mobile.csv")

In [6]:
#Conversion en GeoDataFrame
from shapely import wkt
communes['geometry'] = communes['geometry'].apply(wkt.loads)
communes = gpd.GeoDataFrame(communes, crs = 'epsg:4326')
departements['geometry'] = departements['geometry'].apply(wkt.loads)
departements = gpd.GeoDataFrame(departements, crs = 'epsg:4326')
sans_mobile['geometry'] = sans_mobile['geometry'].apply(wkt.loads)
sans_mobile = gpd.GeoDataFrame(sans_mobile, crs = 'epsg:4326')
defibrillateurs['geometry'] = defibrillateurs['geometry'].apply(wkt.loads)
defibrillateurs = gpd.GeoDataFrame(defibrillateurs, crs = 'epsg:4326')
police['geometry'] = police['geometry'].apply(wkt.loads)
police = gpd.GeoDataFrame(police, crs = 'epsg:4326')
hopitaux['geometry'] = hopitaux['geometry'].apply(wkt.loads)
hopitaux = gpd.GeoDataFrame(hopitaux, crs = 'epsg:4326')
camping['geometry'] = camping['geometry'].apply(wkt.loads)
camping = gpd.GeoDataFrame(camping, crs = 'epsg:4326')

Création de la carte (sortie HTML : Visualisation.html) :

In [7]:
m = communes.plot_bokeh(line_width = 0.1, fill_alpha = 0, legend = "Communes", title = "Sites de Camping en Ile-de-France", show_figure = False)
m.width = 1000
m.height = 700
departements.plot_bokeh(figure = m, line_color = "black", fill_alpha = 0.3, color = "blue", legend = "Départements", show_figure = False)
sans_mobile.plot_bokeh(figure = m, legend = "Pas de Couverture Mobile", color = "grey", line_width = 0.5, show_figure = False)
defibrillateurs.plot_bokeh(figure = m, legend = "Défibrillateurs", size = 0.5, color = "green", show_figure = False)
police.plot_bokeh(figure = m, color = "blue", size = 2, legend = "Police/Gendarmerie", hovertool_columns = ["Nom"], show_figure = False)
hopitaux.plot_bokeh(figure = m, color = "yellow", size = 2, legend = "Hôpitaux/Cliniques",hovertool_columns = ["Nom"], show_figure = False)
camping.plot_bokeh(figure = m, size = 5, legend = "Campings (classés)", category = "Classement", colormap = "Reds", hovertool_columns = ["Nom", "Adresse", "Emplacements", "Classement", "Surface"])

/opt/conda/lib/python3.9/site-packages/pandas_bokeh/geoplot.py:118: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:


Figure(id='1003', ...)

**Choisissez les éléments que vous souhaitez voir affichés sur la carte en cliquant sur la légende**

Il y a 65 sites de camping en Ile-de-France. Sans surprise, la plupart de ces sites sont éparpillés dans la banlieue parisienne et non dans la ville, contrairement aux hôpitaux/cliniques, postes de police/gendarmeries et défibrillateurs qui sont concentrés dans la ville et la petite couronne parisienne et qui sont de moins en moins nombreux plus on s'éloigne de Paris. Si on souhaitait comparer les sites de camping selon leur accessibilité aux établissements de sécurité et de santé, ceux qui sont dans la ville de Paris et dans la peitite couronne seront au même niveau, ayant le plus d'accès à ces services. On observe par ailleurs que ces sites-là sont aussi mieux classés, et donc ont plus d'aménagements en interne comme l'accès à Internet et une permanence 24h/24. En termes de nombre d'emplacements (200-300) et de surface, ceux-là sont aussi les plus grands en Ile-de-France, sans doute à cause de leur proximité à la ville. Les sites plus loins de la ville sont, eux, plus loins des établissements de santé et de sécurité, n'ont pas tous des aménagements tels qu'une connexion Internet ou une permanence 24h/24, et sont plus petits, ayant environ cent emplacements. On a constaté dans le fichier prepare_data que seulement un site de camping n'avait pas une bonne couverture de réseau mobile, les autres sont (heureusement) bien couverts.